In [15]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [16]:
data = [("Juan", 25), ("Ana", 30)]
schema = StructType([
    StructField("name", StringType()),
    StructField("edad", IntegerType())
])

df = spark.createDataFrame(data = data, schema = schema)
df.show()

+----+----+
|name|edad|
+----+----+
|Juan|  25|
| Ana|  30|
+----+----+



In [17]:
df_filtrado = df.filter(df.edad > 28)
df_filtrado.explain()

== Physical Plan ==
*(1) Filter (isnotnull(edad#1) AND (edad#1 > 28))
+- *(1) Scan ExistingRDD[name#0,edad#1]




In [18]:
df_filtrado.show()

+----+----+
|name|edad|
+----+----+
| Ana|  30|
+----+----+



In [19]:
!mkdir -p data

In [20]:
!wget -P data/ https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv

--2025-11-19 08:39:57--  https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 970457 (948K) [text/plain]
Saving to: ‘data/Telco-Customer-Churn.csv’

Telco-Customer-Chur 100%[===================>] 947.71K  --.-KB/s    in 0.06s   

2025-11-19 08:39:57 (16.0 MB/s) - ‘data/Telco-Customer-Churn.csv’ saved [970457/970457]



In [23]:
telco_data = spark.read.csv('./data/Telco-Customer-Churn.csv', header = True, inferSchema = True)
print(telco_data.count())

7043


In [24]:
telco_data.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [55]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

telco_data = telco_data.withColumn('TotalCharges', col("TotalCharges").cast(DoubleType()))
telco_data.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)



In [56]:
df_ivan = telco_data.select('TotalCharges').describe()
df_ivan.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [summary#921, element_at(TotalCharges#908, summary#921, None, false) AS TotalCharges#909]
   +- Generate explode([count,mean,stddev,min,max]), [TotalCharges#908], false, [summary#921]
      +- HashAggregate(keys=[], functions=[count(TotalCharges#881), avg(TotalCharges#881), stddev(TotalCharges#881), min(TotalCharges#881), max(TotalCharges#881)])
         +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=561]
            +- HashAggregate(keys=[], functions=[partial_count(TotalCharges#881), partial_avg(TotalCharges#881), partial_stddev(TotalCharges#881), partial_min(TotalCharges#881), partial_max(TotalCharges#881)])
               +- Project [cast(TotalCharges#117 as double) AS TotalCharges#881]
                  +- FileScan csv [TotalCharges#117] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/data/Telco-Customer-Churn.csv], PartitionFilters: [], PushedFi

In [57]:
df_gem = telco_data.describe(["TotalCharges"])
df_gem.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [summary#962, element_at(TotalCharges#949, summary#962, None, false) AS TotalCharges#950]
   +- Generate explode([count,mean,stddev,min,max]), [TotalCharges#949], false, [summary#962]
      +- HashAggregate(keys=[], functions=[count(TotalCharges#881), avg(TotalCharges#881), stddev(TotalCharges#881), min(TotalCharges#881), max(TotalCharges#881)])
         +- Exchange SinglePartition, ENSURE_REQUIREMENTS, [plan_id=586]
            +- HashAggregate(keys=[], functions=[partial_count(TotalCharges#881), partial_avg(TotalCharges#881), partial_stddev(TotalCharges#881), partial_min(TotalCharges#881), partial_max(TotalCharges#881)])
               +- Project [cast(TotalCharges#117 as double) AS TotalCharges#881]
                  +- FileScan csv [TotalCharges#117] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/data/Telco-Customer-Churn.csv], PartitionFilters: [], PushedFi

In [58]:
telco_data.select('TotalCharges').describe().show()

+-------+------------------+
|summary|      TotalCharges|
+-------+------------------+
|  count|              7032|
|   mean|2283.3004408418697|
| stddev| 2266.771361883145|
|    min|              18.8|
|    max|            8684.8|
+-------+------------------+



In [59]:
telco_data.where(col('TotalCharges').isNull()).count()

11

In [62]:
telco_clean = telco_data.na.drop(subset=["TotalCharges"])
print(telco_clean.where(col('TotalCharges').isNull()).count())
telco_clean.select('TotalCharges').describe().show()

0
+-------+------------------+
|summary|      TotalCharges|
+-------+------------------+
|  count|              7032|
|   mean|2283.3004408418697|
| stddev| 2266.771361883145|
|    min|              18.8|
|    max|            8684.8|
+-------+------------------+



In [64]:
telco_clean.groupBy(['gender', 'Churn']).count().show()

+------+-----+-----+
|gender|Churn|count|
+------+-----+-----+
|  Male|   No| 2619|
|  Male|  Yes|  930|
|Female|   No| 2544|
|Female|  Yes|  939|
+------+-----+-----+



In [73]:
from pyspark.sql.functions import when

telco_clean = telco_clean.withColumn("label", when(col("Churn") == 'Yes', 1).otherwise(0))

In [75]:
telco_clean.select('Churn', 'label').show(5)

+-----+-----+
|Churn|label|
+-----+-----+
|   No|    0|
|   No|    0|
|  Yes|    1|
|   No|    0|
|  Yes|    1|
+-----+-----+
only showing top 5 rows



In [77]:
telco_clean.write.mode('overwrite').parquet('./data/processed')

In [78]:
df_final = spark.read.parquet('./data/processed')
df_final.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)
 |-- label: integer (nullable = true)

